In [10]:
from perplexity import Perplexity
import os
from dotenv import load_dotenv
from pydantic import BaseModel

class format(BaseModel):
    reasoning: str
    in_short: str
load_dotenv()
client = Perplexity()

def invoke(query,format=None):
    response = client.responses.create(
        model="google/gemini-2.5-pro",
        input=query,response_format=format
    )
    return response



In [24]:
from perplexity import Perplexity

class StructuredPerplexity:
    def __init__(self, client, model):
        self.client = client
        self.model = model

    def invoke(self, query: str, schema_model: BaseModel):
        schema = schema_model.model_json_schema()

        prompt = f"""
You are a JSON API.

Rules:
- Output ONLY valid JSON
- No markdown
- No backticks
- No explanation
- No extra keys
- No missing keys
- Values must match types EXACTLY

Schema:
{schema}

User query:
{query}

JSON:
"""

        resp = self.client.responses.create(
            model=self.model,
            input=prompt
        )

        return schema_model.model_validate_json(resp.output_text)


In [25]:
from pydantic import BaseModel
from typing import List

class Answer(BaseModel):
    short_answer: str
    confidence: float
    sources: List[str]


client = Perplexity()
llm = StructuredPerplexity(client, "google/gemini-2.5-flash")

result = llm.invoke(
    "What is the capital of India?",
    Answer
)

print(result.short_answer)


New Delhi


In [27]:
print(result.confidence)

1.0


In [11]:
result = invoke("how are you?",format=format)

TypeError: Object of type ModelMetaclass is not JSON serializable

In [21]:
llm.invoke("hi how are you")

AIMessage(content='"Hi, how are you?" is a common informal English greeting that combines a casual hello with a question about someone\'s well-being.[1][2]\n\n**"Hi"** is one of the most popular greetings for informal situations, typically followed by the person\'s name.[1] It\'s appropriate to use with friends, coworkers, and acquaintances.\n\n**"How are you?"** is a standard follow-up question that opens the conversation.[2] Common responses include:\n\n- "I\'m good, thanks."[2]\n- "Fine, thanks."[2]\n- "I\'m doing okay."[2]\n- "Pretty good. And you?"[2]\n- "Not too bad."[2]\n\nIn U.S. English specifically, it\'s quite common to skip saying "hello" entirely and start directly with a question about how the other person is doing, which makes the conversation get going more quickly.[3]', additional_kwargs={'citations': ['https://preply.com/en/blog/22-useful-english-greetings-for-every-day/', 'https://www.englishalex.com/post/life-beyond-how-are-you-the-most-common-english-greetings-and-

In [10]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
# Choose the best model for your task
model_name = "qwen/qwen3-32b"  # example

llm = ChatGroq(
    model=model_name,
    temperature=0.2,   # adjust as needed
)

response = llm.invoke([{"role": "user", "content": "Explain quantum computing simply."}])
print(response.content)


<think>
Okay, so I need to explain quantum computing in a simple way. Let me start by recalling what I know. Quantum computing uses quantum bits, or qubits, right? Unlike classical bits that are either 0 or 1, qubits can be both at the same time. That's called superposition. But how does that work exactly? Maybe I should compare it to something familiar.

Wait, maybe using the analogy of a light switch. A regular switch is either on or off, but a qubit is like a dimmer switch that can be in between. But that might not capture the full picture. Superposition allows qubits to exist in multiple states simultaneously, which could process a lot of information at once. But how does that translate to computation?

Then there's entanglement. I remember that when qubits are entangled, their states are linked. So changing one affects the other instantly, no matter the distance. That's a key part of quantum computing, but how does that help in solving problems faster?

Also, quantum computers use

In [ ]:
from pydantic import BaseModel,Field

class format(BaseModel):
    thinking: str = Field(...,description="If you are thinking then your thinking should be here.")
    text: str = Field(...,description="The final answer.")


In [14]:
result = llm.invoke("what is the capital of india in 1 word.Reason carefully before answering, but do not include your reasoning in the output.")

In [15]:
result

AIMessage(content='<think>\nOkay, so the user is asking for the capital of India in one word. Let me think. I know that India is a country in South Asia. The capital isn\'t Mumbai or Delhi? Wait, Delhi is often mentioned. But there\'s also New Delhi. Wait, what\'s the difference between Delhi and New Delhi? I think New Delhi is the actual capital city, which is a part of the National Capital Territory of Delhi. So the correct answer should be New Delhi. But sometimes people refer to Delhi as the capital. Hmm. I need to be precise here. Let me confirm. The official capital of India is New Delhi, which is a city within the Delhi region. So the answer should be New Delhi. But the user wants it in one word. Wait, "New Delhi" is two words. Oh, maybe they just want Delhi. But I need to be accurate. The official capital is New Delhi. However, sometimes it\'s written as New Delhi, but maybe in some contexts, it\'s considered a single word. Wait, no, it\'s two words. But maybe the user is expec

In [16]:
import re

def strip_thinking(text: str) -> str:
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

In [17]:
strip_thinking(result.content)

'New Delhi'

In [ ]:
from transformers import pipeline

nlp = pipeline(
    "document-question-answering",
    model="impira/layoutlm-document-qa",
)

nlp(
    "https://templates.invoicehome.com/invoice-template-us-neat-750px.png",
    "What is the invoice number?"
)
# {'score': 0.9943977, 'answer': 'us-001', 'start': 15, 'end': 15}

nlp(
    "https://miro.medium.com/max/787/1*iECQRIiOGTmEFLdWkVIH2g.jpeg",
    "What is the purchase amount?"
)
# {'score': 0.9912159, 'answer': '$1,000,000,000', 'start': 97, 'end': 97}

nlp(
    "https://www.accountingcoach.com/wp-content/uploads/2013/10/income-statement-example@2x.png",
    "What are the 2020 net sales?"
)
# {'score': 0.59147286, 'answer': '$ 3,750', 'start': 19, 'end': 20}


In [6]:
from huggingface_hub import InferenceClient
from dotenv import load_dotenv

load_dotenv()

client = InferenceClient()

response = client.chat_completion(
    model="Qwen/Qwen3-32B",
    messages=[
        {"role": "user", "content": "hello how are you doing"}
    ],

)

print(response.choices[0].message.content)


ValueError: You must provide an api_key to work with groq API or log in with `hf auth login`.